# 🏥 Medical AI Bot - High Accuracy Training (DenseNet121) 🏥

This notebook is designed to train a **highly accurate** medical image classifier.
We use **DenseNet121**, a powerful architecture for X-ray analysis, and automatically download high-quality data from Kaggle.

### 🚀 Step 1: Initialize & Authenticate
1.  Upload your **`kaggle.json`** file below (Get it from your [Kaggle Account](https://www.kaggle.com/account) -> API -> Create New Token).

In [1]:
!pip install -q tf-keras kaggle
import os
from google.colab import files

# Force TensorFlow to use Keras 2 (legacy) format
os.environ['TF_USE_LEGACY_KERAS'] = '1'

# Upload kaggle.json
if not os.path.exists('kaggle.json'):
    print("Upload your kaggle.json file:")
    files.upload()

# Configure Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("✅ Kaggle Configured Successfully!")


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


ModuleNotFoundError: No module named 'google.colab'

### 📥 Step 2: Download & Prepare Data
We will download TWO datasets to ensure we have enough diversity:
1.  **COVID-19 Radiography Database** (Good for COVID/Normal)
2.  **Chest X-Ray Prediction** (Good for Pneumonia)

In [ ]:
print("⏳ Downloading Datasets... Please wait.")

# 1. Download COVID-19 Radiography Database
if not os.path.exists('covid19-radiography-database.zip'):
    !kaggle datasets download -d tawsifurrahman/covid19-radiography-database
    !unzip -q covid19-radiography-database.zip
    print("✅ COVID-19 Database Downloaded.")

# 2. Download Pneumonia Dataset
if not os.path.exists('chest-xray-pneumonia.zip'):
    !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
    !unzip -q chest-xray-pneumonia.zip
    print("✅ Pneumonia Database Downloaded.")

In [ ]:
import shutil
import random
from tqdm import tqdm

# Setup Dataset Directory
DATASET_DIR = 'dataset'
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)
os.makedirs(DATASET_DIR)

CLASSES = ['Normal', 'COVID-19', 'Pneumonia']
for c in CLASSES:
    os.makedirs(os.path.join(DATASET_DIR, c), exist_ok=True)

print("📂 Organizing Data...")

# --- 1. Process COVID-19 Images ---
covid_src = os.path.join('COVID-19_Radiography_Dataset', 'COVID', 'images')
dst = os.path.join(DATASET_DIR, 'COVID-19')
files = [f for f in os.listdir(covid_src) if f.lower().endswith('.png')]
# Use ALL COVID images (usually ~3600)
for f in tqdm(files, desc="Copying COVID"):
    shutil.copy(os.path.join(covid_src, f), os.path.join(dst, f))

# --- 2. Process Normal Images ---
# We limit Normal images to match COVID count roughly to avoid imbalance
normal_src = os.path.join('COVID-19_Radiography_Dataset', 'Normal', 'images')
dst = os.path.join(DATASET_DIR, 'Normal')
files = [f for f in os.listdir(normal_src) if f.lower().endswith('.png')]
selected_files = random.sample(files, min(len(files), 4000)) # improved balance
for f in tqdm(selected_files, desc="Copying Normal"):
    shutil.copy(os.path.join(normal_src, f), os.path.join(dst, f))

# --- 3. Process Pneumonia Images ---
pneum_src = os.path.join('chest_xray', 'train', 'PNEUMONIA')
dst = os.path.join(DATASET_DIR, 'Pneumonia')
files = [f for f in os.listdir(pneum_src) if f.lower().endswith('.jpeg')]
# Pneumonia dataset is large (~3800), take 4000 to match
selected_files = files[:4000] 
for f in tqdm(selected_files, desc="Copying Pneumonia"):
    shutil.copy(os.path.join(pneum_src, f), os.path.join(dst, f))

print("\n✅ Data Preparation Complete!")
for c in CLASSES:
    print(f"   {c}: {len(os.listdir(os.path.join(DATASET_DIR, c)))} images")

### 🧠 Step 3: Build & Train Model (DenseNet121)

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np

# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 64 # Increased batch size for T4 GPU utilization
EPOCHS = 30
DATASET_DIR = "dataset"
CLASSES = ['COVID-19', 'Normal', 'Pneumonia']

print("Loading dataset directly into RAM using high-performance tf.data pipeline...")
AUTOTUNE = tf.data.AUTOTUNE

def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return tf.argmax(parts[-2] == CLASSES)

def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    # RETURN AS UINT8 (0-255) TO SAVE RAM DURING CACHING (2GB vs 9GB)
    return tf.cast(img, tf.uint8)

def process_path(file_path):
    label = get_label(file_path)
    label = tf.one_hot(label, len(CLASSES))
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

# Build dataset
list_ds = tf.data.Dataset.list_files(str(DATASET_DIR + '/*/*'), shuffle=False)
list_ds = list_ds.shuffle(15000, seed=42)
image_count = len(list_ds)

val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

def augment_and_scale(image, label):
    # CONVERT TO FLOAT32 [0, 1] AFTER CACHING
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    return image, label

def scale_only(image, label):
    # CONVERT TO FLOAT32 [0, 1] AFTER CACHING
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label

train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache() # CACHE IN RAM (Now fits easily as uint8)
train_ds = train_ds.shuffle(buffer_size=2000) # Shuffle every epoch
train_ds = train_ds.map(augment_and_scale, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.cache()
val_ds = val_ds.map(scale_only, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

def build_model():
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=IMG_SIZE + (3,)
    )
    
    # Unfreeze the last block for fine-tuning
    base_model.trainable = True
    for layer in base_model.layers[:-40]:
        layer.trainable = False
        
    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()

# Callbacks
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
early_stop = EarlyStopping(monitor='val_accuracy', patience=8, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

print("🚀 Starting High-Speed Training...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

# Save Final Model
model.save('model.h5')
print("✅ Model Saved as 'model.h5'")



### 📊 Step 4: Evaluate & Download

In [ ]:
from google.colab import files
files.download('model.h5')